First, we need to import the module `genealopy`. In this example, since `genealopy.py` is located in the parent folder of the current file, we need to use `sys.path.append(..)` before calling `import genealopy`.

In [ ]:
import sys
sys.path.append('../..')
import genealopy as gp

For this example, we will also need some additional modules.

In [ ]:
import pandas as pd
import unidecode
import random
import datetime

Here we define a very complicated function that generates a random family tree. If you want to see some tricks, don't hesitate to look this code in details or even play with it.

In [ ]:
def generate_random_dynasty(generations: int, incest_ratio: float) -> None:

    def random_name(sex: str = None) -> str:
        names = None
        weights = None
        if sex == "m": table = pd.read_csv("first_names_m.csv", sep=";")
        if sex == "f": table = pd.read_csv("first_names_f.csv", sep=";")
        if sex == None: table = pd.read_csv("last_names.csv", sep=";")

        names = table["name"].values
        weights = table["frequency"].values
        
        name = unidecode.unidecode(random.choices(names, weights=weights)[0])
        if len(name) > 2:
            return name
        else:
            return random_name(sex=sex)
    
    dynasty = gp.Genealogy()

    initial_last_name = "Leuenberger"

    individual_ids = []
    family_ids = []

    individual_ids.append(dynasty.create_individual(
        first_name="Friedrich",
        last_name=initial_last_name,
        sex="M",
        birth_date=datetime.date(1800, 10, 3),
        death_date=datetime.date(1900, 10, 3),
    ))
        
    for generation in (range(generations)):
        
        # Adding families to individuals
        
        for individual_id in individual_ids:
            individual_obj = dynasty.__individuals__.query("id == @individual_id").iloc[0]
            
            # Adding a family for future parents
            
            if individual_obj["family_as_child_index"] is None:
                if individual_obj["last_name"] == initial_last_name or random.random() < 0.25:
                    family_ids.append(dynasty.create_family(
                        marriage_date=datetime.date(individual_obj["birth_date"].year+random.randint(-2, 2), random.randint(1, 12), random.randint(1, 28)),
                    ))
                    dynasty.add_individual_to_family(individual_id=individual_id, family_id=family_ids[-1], role="child")
            
            # Adding a family for future children
            
            if individual_obj["families_as_husband_indices"] == [] and dynasty.__individuals__.query("id == @individual_id").iloc[0]["families_as_wife_indices"] == []:
                if random.choice([True, False]):
                    if individual_obj["sex"] == "M":
                        family_ids.append(dynasty.create_family(
                            marriage_date=datetime.date(individual_obj["birth_date"].year+random.randint(20, 30), random.randint(1, 12), random.randint(1, 28)),
                        ))
                        dynasty.add_individual_to_family(individual_id=individual_id, family_id=family_ids[-1], role="husband")
                    else:
                        family_ids.append(dynasty.create_family(
                            marriage_date=datetime.date(individual_obj["birth_date"].year+random.randint(18, 28), random.randint(1, 12), random.randint(1, 28)),
                        ))
                        dynasty.add_individual_to_family(individual_id=individual_id, family_id=family_ids[-1], role="wife")
        
        # Populating families with individuals
        
        for family_id in family_ids:
            family_obj = dynasty.__families__.query("id == @family_id").iloc[0]
            
            # Getting family name
            
            last_name = random_name()
            if family_obj["husband_index"] is not None:
                husband_index = family_obj["husband_index"]
                husband_obj = dynasty.__individuals__.query("id == @husband_index")
                last_name = husband_obj["last_name"].values[0]
            elif len(family_obj["children_indices"]) > 0:
                child_index = family_obj["children_indices"][0]
                child_obj = dynasty.__individuals__.query("id == @child_index")
                last_name = child_obj["last_name"].values[0]
            
            # Adding father
            
            if family_obj["husband_index"] is None:
                birth_date = datetime.date(family_obj["marriage_date"].year-random.randint(20, 30), random.randint(1, 12), random.randint(1, 28))
                potential_fathers = dynasty.__individuals__[
                    (dynasty.__individuals__["sex"] == "M") & # looking for all the mens
                    (dynasty.__individuals__["families_as_husband_indices"].apply(lambda x: len(x) == 0)) & # looking for all the bachelors
                    (dynasty.__individuals__["birth_date"].apply(lambda d: abs((d - birth_date).days / 365.25) <= 3) # looking for all people born within 3 years of the original targetted birth date
                )]
                if len(potential_fathers) > 0 and random.random() < incest_ratio:
                    father_index = random.choice(potential_fathers.index)
                    father_id = potential_fathers.loc[father_index]["id"]
                    dynasty.add_individual_to_family(individual_id=father_id, family_id=family_id, role="husband")
                else:
                    individual_ids.append(dynasty.create_individual(
                        first_name=random_name("m"),
                        last_name=last_name,
                        sex="M",
                        birth_date=birth_date,
                        death_date=birth_date+datetime.timedelta(days=365.25 * (random.random() * 40 + 50)),
                    ))
                    dynasty.add_individual_to_family(individual_id=individual_ids[-1], family_id=family_id, role="husband")
                
            # Adding mother
                
            if family_obj["wife_index"] is None:
                birth_date = datetime.date(family_obj["marriage_date"].year-random.randint(18, 28), random.randint(1, 12), random.randint(1, 28))
                potential_mothers = dynasty.__individuals__[(
                    dynasty.__individuals__["sex"] == "F") & # looking for all the mens
                    (dynasty.__individuals__["families_as_wife_indices"].apply(lambda x: len(x) == 0)) & # looking for all the bachelors
                    (dynasty.__individuals__["birth_date"].apply(lambda d: abs((d - birth_date).days / 365.25) <= 3) # looking for all people born within 3 years of the original targetted birth date
                )]
                if len(potential_mothers) > 0 and random.random() < incest_ratio:
                    mother_index = random.choice(potential_mothers.index)
                    mother_id = potential_mothers.loc[mother_index]["id"]
                    dynasty.add_individual_to_family(individual_id=mother_id, family_id=family_id, role="wife")
                else:
                    individual_ids.append(dynasty.create_individual(
                        first_name=random_name("f"),
                        last_name=random_name(),
                        sex="F",
                        birth_date=birth_date,
                        death_date=birth_date+datetime.timedelta(days=365.25 * (random.random() * 50 + 50)),
                    ))
                    dynasty.add_individual_to_family(individual_id=individual_ids[-1], family_id=family_id, role="wife")
                
            # Adding children
                
            if len(family_obj["children_indices"]) <= 1:
                if last_name == initial_last_name:
                    child_count = round(random.expovariate(1/2.0))
                else:
                    child_count = round(random.expovariate(1/0.5))
                for i in range(child_count - len(family_obj["children_indices"])):
                    birth_date = None
                    if len(family_obj["children_indices"]) > 0:
                        
                        # We look the birth date of the last child if applicable.
                        # We then take its birth date, and add a random time between
                        # 9 months and a certain number of years. This number of years
                        # depends on the total number of children. The more children
                        # there are, the less time there is between each child.
                        # For 2 children, the max time between two children is 20/(2-1) = 20 years.
                        # For 3 children, the max time between two children is 20/(3-1) = 10 years.
                        # For 4 children, the max time between two children is 20/(3-1) ~ 6 years.
                        # Therefore, in the worst case, the mother has her last child at around 40~50 years old,
                        # depending on when she got her first child (at around 20~30 years old).
                        
                        last_child_index = family_obj["children_indices"][-1]
                        last_child_obj = dynasty.__individuals__.query("id == @last_child_index")
                        last_child_birth_date = last_child_obj["birth_date"].values[0]
                        birth_date = last_child_birth_date + datetime.timedelta(days=90 * 3 + random.random() * (365 * 20 / (child_count - 1)))
                    else:
                        birth_date = datetime.date(family_obj["marriage_date"].year+random.randint(-2, 10), random.randint(1, 12), random.randint(1, 28))
                    sex = random.choice(["M", "F"])
                    if sex == "M":
                        individual_ids.append(dynasty.create_individual(
                            first_name=random_name("m"),
                            last_name=last_name,
                            sex="M",
                            birth_date=birth_date,
                            death_date=birth_date+datetime.timedelta(days=365.25 * (random.random() * 40 + 50)),
                        ))
                    else:
                        individual_ids.append(dynasty.create_individual(
                            first_name=random_name("f"),
                            last_name=last_name,
                            sex="F",
                            birth_date=birth_date,
                            death_date=birth_date + datetime.timedelta(days=365.25 * (random.random() * 50 + 50)),
                        ))
                    dynasty.add_individual_to_family(individual_id=individual_ids[-1], family_id=family_id, role="child")

        print(f"{generation+1}/{generations} : {len(individual_ids)} individuals in the dynasty.")
    main_family_count = len(dynasty.__individuals__.query("last_name==@initial_last_name"))
    print(f"{main_family_count} '{initial_last_name}' created.")
    print(f"{main_family_count / len(individual_ids) * 100:.0f}% of the individuals have the last name '{initial_last_name}'.")

    with open("output.ged", "w") as f:
        f.write(dynasty.gedcom())

Here we call this function.

In [ ]:
generate_random_dynasty(generations=5, incest_ratio=0.5)